<a href="https://colab.research.google.com/github/Loga19818eeanvesh/Natural_Language_Processing-Assignments/blob/main/Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import argparse
import numpy as np
import random

In [10]:
def softmax(x):
    """Compute the softmax function for each row of the input x.
    It is crucial that this function is optimized for speed because
    it will be used frequently in later code. 
    Arguments:
    x -- A D dimensional vector or N x D dimensional numpy matrix.
    Return:
    x -- You are allowed to modify x in-place
    """
    orig_shape = x.shape

    if len(x.shape) > 1:
        # Matrix
        tmp = np.max(x, axis=1)
        x -= tmp.reshape((x.shape[0], 1))
        x = np.exp(x)
        tmp = np.sum(x, axis=1)
        x /= tmp.reshape((x.shape[0], 1))
    else:
        # Vector
        tmp = np.max(x)
        x -= tmp
        x = np.exp(x)
        tmp = np.sum(x)
        x /= tmp

    assert x.shape == orig_shape
    return x

In [11]:
def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    Arguments:
    x -- A scalar or numpy array.
    Return:
    s -- sigmoid(x)
    """

    ### YOUR CODE HERE (~1 Line)
    s = 1 / (1 + np.exp(-x))
    ### END YOUR CODE

    return s

In [12]:
def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Naive Softmax loss & gradient function for word2vec models
    Implement the naive softmax loss and gradients between a center word's 
    embedding and an outside word's embedding. This will be the building block
    for our word2vec models. For those unfamiliar with numpy notation, note 
    that a numpy ndarray with a shape of (x, ) is a one-dimensional array, which
    you can effectively treat as a vector with length x.
    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    in shape (word vector length, )
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors is
                    in shape (num words in vocab, word vector length) 
                    for all words in vocab (tranpose of U in the pdf handout)
    dataset -- needed for negative sampling, unused here.
    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     in shape (word vector length, )
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    in shape (num words in vocab, word vector length) 
                    (dJ / dU)
    """

    ### YOUR CODE HERE (~6-8 Lines)

    ### Please use the provided softmax function (imported earlier in this file)
    ### This numerically stable implementation helps you avoid issues pertaining
    ### to integer overflow. 
    
    gradOutsideVecs = np.zeros_like(outsideVectors)
    # obtain y_hat (i.e., the conditional probability distribution p(O = o | C = c))
    # by taking vector dot products and applying softmax
    y_hat = softmax(np.dot(outsideVectors, centerWordVec)) # (N,) N x 1
    # can also get y_hat in a single line: y_hat = softmax(outsideVectors @ centerWordVec)

    # for a single pair of words c and o, the loss is given by:
    # J(v_c, o, U) = -log P(O = o | C = c) = -log [y_hat[o]]
    loss = -np.log(y_hat[outsideWordIdx])

    # grad calc
    # generate the ground-truth one-hot vector, [..., 0, outsideWordIdx=1, 0, ...]
    y = np.zeros_like(y_hat)
    y[outsideWordIdx] = 1
    # can also get loss as -np.dot(y, np.log(y_hat))    
    
    gradCenterVec = np.dot(y_hat - y, outsideVectors) # inner product results in a scalar
    # or gradCenterVec = np.dot(outsideVectors.T, y_hat - y)
    
    gradOutsideVecs = np.dot((y_hat - y)[:, np.newaxis], centerWordVec[np.newaxis, :]) 
    
    # sanity check the dimensions
    assert gradCenterVec.shape == centerWordVec.shape
    assert gradOutsideVecs.shape == outsideVectors.shape  

    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

In [13]:
def getNegativeSamples(outsideWordIdx, dataset, K):
    """ Samples K indexes which are not the outsideWordIdx """

    negSampleWordIndices = [None] * K
    for k in range(K):
        newidx = dataset.sampleTokenIdx()
        while newidx == outsideWordIdx:
            newidx = dataset.sampleTokenIdx()
        negSampleWordIndices[k] = newidx
    return negSampleWordIndices


In [7]:
def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    """ Negative sampling loss function for word2vec models
    Implement the negative sampling loss and gradients for a centerWordVec
    and a outsideWordIdx word vector as a building block for word2vec
    models. K is the number of negative samples to take.
    Note: The same word may be negatively sampled multiple times. For
    example if an outside word is sampled twice, you shall have to
    double count the gradient with respect to this word. Thrice if
    it was sampled three times, and so forth.
    Arguments/Return Specifications: same as naiveSoftmaxLossAndGradient
    """

    # Negative sampling of words is done for you. Do not modify this if you
    # wish to match the autograder and receive points!
    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    ### YOUR CODE HERE (~10 Lines)

    ### Please use your implementation of sigmoid in here.
    
    gradOutsideVecs = np.zeros(outsideVectors.shape)
    
    # Calculate the first term
    y_hat = sigmoid(np.dot(outsideVectors[outsideWordIdx], centerWordVec))
    loss = -np.log(y_hat)
    
    gradCenterVec = np.dot(y_hat - 1, outsideVectors[outsideWordIdx])
    gradOutsideVecs[outsideWordIdx] = np.dot(y_hat - 1, centerWordVec)

    # Calculate the second term
    for i in range(K):
        w_k = indices[i+1]
        y_k_hat = sigmoid(-np.dot(outsideVectors[w_k], centerWordVec))
        loss += -np.log(y_k_hat)
        gradOutsideVecs[w_k] += np.dot(1.0 - y_k_hat, centerWordVec)
        gradCenterVec += np.dot(1.0 - y_k_hat, outsideVectors[w_k])

    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

In [8]:
def skipgram(currentCenterWord, windowSize, outsideWords, word2Ind,
             centerWordVectors, outsideVectors, dataset,
             word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    """ Skip-gram model in word2vec
    Implement the skip-gram model in this function.
    Arguments:
    currentCenterWord -- a string of the current center word
    windowSize -- integer, context window size
    outsideWords -- list of no more than 2*windowSize strings, the outside words
    word2Ind -- a dictionary that maps words to their indices in
              the word vector list
    centerWordVectors -- center word vectors (as rows) is in shape 
                        (num words in vocab, word vector length) 
                        for all words in vocab (V in pdf handout)
    outsideVectors -- outside vectors is in shape 
                        (num words in vocab, word vector length) 
                        for all words in vocab (transpose of U in the pdf handout)
    word2vecLossAndGradient -- the loss and gradient function for
                               a prediction vector given the outsideWordIdx
                               word vectors, could be one of the two
                               loss functions you implemented above.
    Return:
    loss -- the loss function value for the skip-gram model
            (J in the pdf handout)
    gradCenterVec -- the gradient with respect to the center word vector
                     in shape (word vector length, )
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    in shape (num words in vocab, word vector length) 
                    (dJ / dU)
    """

    loss = 0.0
    gradCenterVecs = np.zeros(centerWordVectors.shape)
    gradOutsideVectors = np.zeros(outsideVectors.shape)

    ### YOUR CODE HERE (~8 Lines)
    
    # skip-gram model predicts outside words from the center word
    
    # get center word vec first from currentCenterWord
    centerWordIdx = word2Ind[currentCenterWord]
    centerWordVec = centerWordVectors[centerWordIdx]

    for outsideWord in outsideWords:
        outsideWordIdx = word2Ind[outsideWord]
        stepLoss, gradCenter, gradOutside = word2vecLossAndGradient(centerWordVec,
                                                                    outsideWordIdx,
                                                                    outsideVectors,
                                                                    dataset)

        loss += stepLoss
        gradCenterVecs[centerWordIdx] += gradCenter
        gradOutsideVectors += gradOutside    

    ### END YOUR CODE
    
    return loss, gradCenterVecs, gradOutsideVectors
